In [ ]:
# installing spacy word2vec model

!pip install spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.1 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=5c840fd3eb9c79474fb47ac9205d85e988cbdb8c3e98ac34bafb5be2c578ff4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-b2igfutk/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
def get_vec(text):
  text2 = nlp(text)
  return text2.vector
  


In [ ]:
import pandas as pd
df_train = pd.read_csv("training_task_a_tweets.csv")
df_test = pd.read_csv("test_a.csv")

In [ ]:
df_train.head()

,id,in_reply_to_status_id_str,text,user.verified,user.followers_count,retweet_count,favorite_count,entities.hashtags,entities.urls,label
0,552785374507175936,552783667052167168,MT @euronews France: 10 dead after shooting at...,False,1828,1,2,"[{'indices': [70, 83], 'text': 'CharlieHebdo'}]",[],comment
1,552786226546495488,552785374507175936,@j0nathandavis They who? Stupid and partial op...,False,405,0,0,[],[],deny
2,552789345628069888,552786226546495488,"@nanoSpawn Socialists, Antisemites, anti zioni...",False,1828,0,0,[],[],comment
3,552792862610694144,552783667052167168,@euronews @TradeDesk_Steve A French crime of p...,False,26,0,0,[],[],query
4,552943855021330432,552783667052167168,"@euronews LOL. 5 million Muslims in France, wh...",False,14,0,0,[],[],comment


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing

In [ ]:
import nltk
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df_train.label.unique()

array(['comment', 'deny', 'query', 'support'], dtype=object)

In [ ]:
df_train["text"]=df_train["text"].str.lower()
df_test["text"]=df_test["text"].str.lower()

In [ ]:
def getLemmText(text):
 tokens=word_tokenize(text)
 lemmatizer = WordNetLemmatizer()
 tokens=[lemmatizer.lemmatize(word) for word in tokens]
 return ' '.join(tokens)
def stop_words(text):
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  return text
def getStemmText(text):
 tokens=word_tokenize(text)
 ps = PorterStemmer()
 tokens=[ps.stem(word) for word in tokens]
 return ' '.join(tokens)

In [ ]:
df_train['text'] = list(map(stop_words,df_train['text']))
df_train['text'] = list(map(getLemmText,df_train['text']))
df_train['text'] = list(map(getStemmText,df_train['text']))


df_test['text'] = list(map(stop_words,df_test['text']))
df_test['text'] = list(map(getLemmText,df_test['text']))
df_test['text'] = list(map(getStemmText,df_test['text']))

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train.label.values)
df_test['label'] = label_encoder.fit_transform(df_test.label.values)

In [ ]:
print(df_train['label'].value_counts())

0    2896
3     633
2     358
1     335
Name: label, dtype: int64


In [ ]:
df_train.drop(df_train.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
df_test.drop(df_test.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)

In [ ]:
df_train.head()

,text,label
0,mt euronew franc 10 dead shoot hq satir weekli...,0
1,j0nathandavi stupid partial opinion like one a...,1
2,nanospawn socialist antisemit anti zionist usu...,0
3,euronew tradedesk_stev french crime passion an...,2
4,euronew lol 5 million muslim franc disgrac fre...,0


In [ ]:
df_train['text'] = df_train['text'].apply(lambda x : get_vec(x))
df_test['text'] = df_test['text'].apply(lambda x : get_vec(x))

In [ ]:
df_train.head()

,text,label
0,"[-0.27525347, 0.059543, 0.06431934, 0.02177257...",0
1,"[-0.16507034, -0.04077711, -0.14735556, 0.0035...",1
2,"[-0.21171144, -0.14161286, 0.10437315, 0.19838...",0
3,"[0.014992446, -0.017640773, -0.0073455563, -0....",2
4,"[-0.23226844, -0.0009961469, 0.036709342, -0.0...",0


In [ ]:
x_train = df_train['text'].to_numpy()
x_test = df_test['text'].to_numpy()
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)

In [ ]:
x_train.shape

(4222, 1)

In [ ]:
x_train[0]

array([array([-0.27525347,  0.059543  ,  0.06431934,  0.02177258,  0.09346569,
              -0.16034956, -0.06996989, -0.07239709, -0.01079598,  0.57832754,
              -0.16123123, -0.0301227 , -0.05116794,  0.00676735,  0.04161347,
              -0.03854943, -0.18033516,  0.45451587, -0.048151  , -0.00740294,
               0.22198617,  0.11762583,  0.03409126,  0.00148382,  0.009736  ,
               0.08137332, -0.16460742,  0.01132412,  0.07497165,  0.25618014,
              -0.01761876,  0.1063835 , -0.04746253,  0.02019235,  0.09691288,
              -0.01245065,  0.07870041,  0.01582235, -0.12185784,  0.19130592,
              -0.10015474, -0.02462111,  0.1314986 , -0.07724071,  0.08610795,
               0.04449995,  0.18791999, -0.07221201, -0.05504159,  0.14019448,
              -0.10962977,  0.09192318, -0.10473929, -0.00574664, -0.04945288,
               0.0383323 , -0.06412177, -0.06679235,  0.23021807,  0.01579382,
              -0.15659352,  0.01847353, -0.09672987,

In [ ]:
x_train = np.concatenate(np.concatenate(x_train,axis = 0),axis = 0).reshape(-1,300)
x_test = np.concatenate(np.concatenate(x_test,axis = 0),axis = 0).reshape(-1,300)

In [ ]:
x_train.shape

(4222, 300)

In [ ]:
y_train = df_train['label']
y_test = df_test['label']

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
 
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(x_train, y_train)
prediction_linear = classifier_linear.predict(x_test)

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print(metrics.confusion_matrix(y_test,prediction_linear),"\n")
print(metrics.accuracy_score(y_test,prediction_linear))

print(precision_score(y_test, prediction_linear , average="macro"))
print(recall_score(y_test, prediction_linear , average="macro"))
print(f1_score(y_test, prediction_linear , average="macro"))

[[771   0   0   0]
 [ 92   0   0   0]
 [ 56   0   0   0]
 [ 91   0   0   0]] 

0.7633663366336634
0.19084158415841584
0.25
0.21645143177989892


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**************"Implementing Subtask B using word2vec*********"

In [ ]:
import pandas as pd
df2_train = pd.read_csv("training_task_b_source_tweets.csv")
df2_test = pd.read_csv("test_task_b_tweets.csv")

In [ ]:
df2_train.head()

,id,in_reply_to_status_id_str,text,user.verified,user.followers_count,retweet_count,favorite_count,entities.hashtags,entities.urls,label
0,552783667052167168,NaN,France: 10 people dead after shooting at HQ of...,True,129573,486,38,"[{'indices': [74, 87], 'text': 'CharlieHebdo'}]",[],true
1,552785375161499649,NaN,BREAKING: 10 reportedly shot dead at Paris HQ ...,True,972167,128,5,[],"[{'url': 'http://t.co/5F1DOwzoCQ', 'indices': ...",true
2,552791196247269378,NaN,BREAKING: At least 10 killed in shooting at Fr...,True,3029912,295,78,[],"[{'url': 'http://t.co/mnAeA7j7fY', 'indices': ...",true
3,552791578893619200,NaN,Eleven dead in shooting at Paris offices of sa...,True,3091451,338,28,[],"[{'url': 'http://t.co/gOtSqbwROc', 'indices': ...",true
4,552792544132997121,NaN,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,True,973212,203,32,[],"[{'url': 'http://t.co/kjW3uwCV0f', 'indices': ...",true


In [ ]:
df2_train.label.unique()

array(['true', 'false', 'unverified'], dtype=object)

In [ ]:
df2_train["text"]=df2_train["text"].str.lower()
df2_test["text"]=df2_test["text"].str.lower()

In [ ]:
df2_train['text'] = list(map(stop_words,df2_train['text']))
df2_train['text'] = list(map(getLemmText,df2_train['text']))
df2_train['text'] = list(map(getStemmText,df2_train['text']))


df2_test['text'] = list(map(stop_words,df2_test['text']))
df2_test['text'] = list(map(getLemmText,df2_test['text']))
df2_test['text'] = list(map(getStemmText,df2_test['text']))

In [ ]:
label_encoder = preprocessing.LabelEncoder()
df2_train['label'] = label_encoder.fit_transform(df2_train.label.values)
df2_test['label'] = label_encoder.fit_transform(df2_test.label.values)

In [ ]:
print(df2_train['label'].value_counts())

1    137
2     98
0     62
Name: label, dtype: int64


In [ ]:
df2_train.drop(df2_train.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)
df2_test.drop(df2_test.columns[[0, 1,3,4,5,6,7,8]], axis = 1, inplace = True)

In [ ]:
df2_train.head()

,text,label
0,franc 10 peopl dead shoot hq satir weekli news...,1
1,break 10 reportedli shot dead pari hq french w...,1
2,break least 10 kill shoot french satir newspap...,1
3,eleven dead shoot pari offic satir magazin cha...,1
4,break charli hebdo latest 11 dead 10 wound fiv...,1


In [ ]:
df2_train['text'] = df2_train['text'].apply(lambda x : get_vec(x))
df2_test['text'] = df2_test['text'].apply(lambda x : get_vec(x))

In [ ]:
df2_train.head()

,text,label
0,"[-0.15615125, 0.12408118, 0.08153687, -0.06293...",1
1,"[-0.16451295, 0.05102164, 0.005728279, 0.01152...",1
2,"[-0.23629186, 0.07596141, -0.06191148, 0.02933...",1
3,"[-0.10541529, -0.01795443, 0.0015363544, 0.001...",1
4,"[-0.087843895, 0.09026472, 0.0024326318, 0.103...",1


In [ ]:
x_train = df2_train['text'].to_numpy()
x_test = df2_test['text'].to_numpy()
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)

In [ ]:
x_train.shape

(297, 1)

In [ ]:
x_train = np.concatenate(np.concatenate(x_train,axis = 0),axis = 0).reshape(-1,300)
x_test = np.concatenate(np.concatenate(x_test,axis = 0),axis = 0).reshape(-1,300)

In [ ]:
x_train.shape

(297, 300)

In [ ]:
y_train = df2_train['label']
y_test = df2_test['label']

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
 
classifier_linear = svm.SVC(kernel='linear',probability=True)
classifier_linear.fit(x_train, y_train)
prediction_linear = classifier_linear.predict(x_test)

In [ ]:
prediction_prob = classifier_linear.predict_proba(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
import math
y_true = [np.argmax(val) for val in y_test]
conf = []
for i in range(len(y_test)):
  conf.append(max(prediction_prob[i]))
RMSE = mean_squared_error([1]*len(conf), conf,squared = False)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.47310753463841804


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print(metrics.confusion_matrix(y_test,prediction_linear),"\n")
print(metrics.accuracy_score(y_test,prediction_linear))

print(precision_score(y_test, prediction_linear , average="macro"))
print(recall_score(y_test, prediction_linear , average="macro"))
print(f1_score(y_test, prediction_linear , average="macro"))

[[ 1 17 12]
 [ 2 10 10]
 [ 0  2  2]] 

0.23214285714285715
0.25383141762452105
0.32929292929292925
0.19854002206943386
